## Imports

In [2]:
import os
import pandas as pd
import csv
from transformers import pipeline
from datasets import Dataset

## Seleccionar solo las reseñas clasificadas como válidas y almacenarlas 

In [3]:
inputFolderPath = "../1. Data/3. Google Maps Reviews Without Emojis"
ouputFolderPath = "../1. Data/5. Classified Reviews"

modelPath = "../../../1. Models/Classifier/Results1"

In [4]:
BATCH_SIZE  = 32
classifier = pipeline(
    "text-classification",
    model = modelPath,
    tokenizer = modelPath,
    device = 0,              
    batch_size = BATCH_SIZE,
    truncation = True,
    max_length = 512,
    return_all_scores = False # Only the most probable label
)

/home/ibon/Programas/anaconda3/envs/TFG/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Procesar todos los csvs

In [5]:
def load_reviews(path):
    with open(path, encoding="utf-8", newline="") as f:
        reader = csv.reader(f, quotechar='"', skipinitialspace=True)
        rows   = [row[0] for row in reader if row]
    # Si la primera fila era la cabecera "Review", la descartamos
    if rows and rows[0].strip().lower() == "review":
        rows = rows[1:]
    return pd.DataFrame({"Review": rows})

In [6]:
VALID_LABEL = "LABEL_1"

for station in os.listdir(inputFolderPath):

    if not station.lower().endswith(".csv"):
        continue

    # Import the data
    inputPath = os.path.join(inputFolderPath, station)
    df = load_reviews(inputPath)
    texts = df["Review"].astype(str).tolist()

    # Obtain the classification
    results = classifier(texts)
    labels = [result["label"] for result in results]

    # Filter only the valid reviews
    mask = [label == VALID_LABEL for label in labels]
    dfValid = df.loc[mask]

    # Save the valid reviews
    outPath = os.path.join(ouputFolderPath, station)
    dfValid.to_csv(outPath, index = False)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
